In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pprint
from uuid import UUID
import gspread_dataframe as gsdf

In [2]:
def activate():
    scope = [
        'https://www.googleapis.com/auth/drive',
        'https://www.googleapis.com/auth/drive.file'
        ]
    key_file_name = 'main-webserver-clientkey.json'
    creds = ServiceAccountCredentials.from_json_keyfile_name(f'../keys/{key_file_name}',scope)
    return gspread.authorize(creds)

In [3]:
def read_latest_recos():
    # Writes a set of article Recommendations to the google sheet
    
    gs_client = activate()
    file = gs_client.open_by_key('1X1nGWEdD2ExW1Z9i1cX5gCpZjYmAFc14GzmoKmnJYU0')  # All writes are to this specific file
    worksheet = file.worksheet('Recos')  # Recos are written to this sheet

    recos = gsdf.get_as_dataframe(worksheet=worksheet, header=0, nrows=None)
    last_date = recos['art_date'].max()
    recos = recos[recos['art_date'] == last_date]
    recos.drop(['uuid', 'user_review'], axis=1, inplace=True)
    
    worksheet.client.session.close()
    return recos